https://medium.com/@rushing_andrei/function-calling-with-open-source-llms-594aa5b3a304

https://github.com/PromptEngineer48/Function-Calling-Ollama-Llama-3.1/blob/main/app.py

https://github.com/samwit/agent_tutorials/blob/main/ollama_agents/llama3_local/llama3_ollama_functions.py

In [6]:
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

llm = ChatGroq(model='llama-3.2-90b-vision-preview')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001CF9E465F90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001CF9DE61D50>, model_name='llama-3.2-90b-vision-preview', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [25]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

@tool
def subtract(a: int, b: int) -> int:
    """Subtracts b from a.

    Args:
        a: first int
        b: second int
    """
    return a - b

tools = [add, multiply, subtract]

In [58]:
print(add)

name='add' description='Adds a and b.\n\n    Args:\n        a: first int\n        b: second int' args_schema=<class 'langchain_core.utils.pydantic.add'> func=<function add at 0x000001CF9E63F4C0>


In [26]:
llm_with_tools = llm.bind_tools(tools)
llm_with_tools

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001CF9E465F90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001CF9DE61D50>, model_name='llama-3.2-90b-vision-preview', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': 'Adds a and b.\n\n    Args:\n        a: first int\n        b: second int', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'multiply', 'description': 'Multiplies a and b.\n\n    Args:\n        a: first int\n        b: second int', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'subtract', 'description': 'Subtracts b from a.\n\n    Args:\n        a: first int\n    

In [27]:
ai_msg = llm_with_tools.invoke("What is (121+5)*9 -5 ?")

ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5dy8', 'function': {'arguments': '{"a": "126", "b": "9"}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_ahgw', 'function': {'arguments': '{"a": "1134", "b": "5"}', 'name': 'subtract'}, 'type': 'function'}, {'id': 'call_2xa6', 'function': {'arguments': '{"a": "121", "b": "5"}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 434, 'total_tokens': 492, 'completion_time': 0.209169079, 'prompt_time': 0.048858069, 'queue_time': 0.05735371000000001, 'total_time': 0.258027148}, 'model_name': 'llama-3.2-90b-vision-preview', 'system_fingerprint': 'fp_07b97e5459', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4e7a752b-7706-4e49-9641-87e5517eee85-0', tool_calls=[{'name': 'multiply', 'args': {'a': '126', 'b': '9'}, 'id': 'call_5dy8', 'type': 'tool_call'}, {'name': 'subtract', 'args': {'a': '1134', 'b': '5'}, 'id': 'call_ahgw', 'type': 'tool_c

In [28]:
ai_msg.tool_calls

[{'name': 'multiply',
  'args': {'a': '126', 'b': '9'},
  'id': 'call_5dy8',
  'type': 'tool_call'},
 {'name': 'subtract',
  'args': {'a': '1134', 'b': '5'},
  'id': 'call_ahgw',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': '121', 'b': '5'},
  'id': 'call_2xa6',
  'type': 'tool_call'}]

In [29]:
llm_with_tools.to_json()['kwargs']['kwargs']['tools']

[{'type': 'function',
  'function': {'name': 'add',
   'description': 'Adds a and b.\n\n    Args:\n        a: first int\n        b: second int',
   'parameters': {'properties': {'a': {'type': 'integer'},
     'b': {'type': 'integer'}},
    'required': ['a', 'b'],
    'type': 'object'}}},
 {'type': 'function',
  'function': {'name': 'multiply',
   'description': 'Multiplies a and b.\n\n    Args:\n        a: first int\n        b: second int',
   'parameters': {'properties': {'a': {'type': 'integer'},
     'b': {'type': 'integer'}},
    'required': ['a', 'b'],
    'type': 'object'}}},
 {'type': 'function',
  'function': {'name': 'subtract',
   'description': 'Subtracts b from a.\n\n    Args:\n        a: first int\n        b: second int',
   'parameters': {'properties': {'a': {'type': 'integer'},
     'b': {'type': 'integer'}},
    'required': ['a', 'b'],
    'type': 'object'}}}]

In [30]:
from langchain_core.messages import HumanMessage, ToolMessage

query = 'what is 4+3-5 and then multiply the results by 5'
messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(query)
print("ai_msg:",ai_msg)
print("ai_msg_tool:",ai_msg.tool_calls)
messages.append(ai_msg)
print("mess:",messages)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply,"subtract":subtract}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

ai_msg: content='' additional_kwargs={'tool_calls': [{'id': 'call_kxk7', 'function': {'arguments': '{"a": 4, "b": 3}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_r99x', 'function': {'arguments': '{"a": 7, "b": 5}', 'name': 'subtract'}, 'type': 'function'}, {'id': 'call_08jq', 'function': {'arguments': '{"a": 2, "b": 5}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 439, 'total_tokens': 493, 'completion_time': 0.197488276, 'prompt_time': 0.057559949, 'queue_time': 0.05665605, 'total_time': 0.255048225}, 'model_name': 'llama-3.2-90b-vision-preview', 'system_fingerprint': 'fp_07b97e5459', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-9d9bc4da-b3a6-43e3-b4e2-86e8da6bbb68-0' tool_calls=[{'name': 'add', 'args': {'a': 4, 'b': 3}, 'id': 'call_kxk7', 'type': 'tool_call'}, {'name': 'subtract', 'args': {'a': 7, 'b': 5}, 'id': 'call_r99x', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 2, 'b': 5}

[HumanMessage(content='what is 4+3-5 and then multiply the results by 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kxk7', 'function': {'arguments': '{"a": 4, "b": 3}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_r99x', 'function': {'arguments': '{"a": 7, "b": 5}', 'name': 'subtract'}, 'type': 'function'}, {'id': 'call_08jq', 'function': {'arguments': '{"a": 2, "b": 5}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 439, 'total_tokens': 493, 'completion_time': 0.197488276, 'prompt_time': 0.057559949, 'queue_time': 0.05665605, 'total_time': 0.255048225}, 'model_name': 'llama-3.2-90b-vision-preview', 'system_fingerprint': 'fp_07b97e5459', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9d9bc4da-b3a6-43e3-b4e2-86e8da6bbb68-0', tool_calls=[{'name': 'add', 'args': {'a': 4, 'b': 3}, 'id': 'call_kxk7', 'type': 'tool_call'}, {'name'

In [31]:
final_response = llm.invoke(messages)
print(final_response)
print(final_response.content)

content='First, 4 + 3 = 7. \nThen, 7 - 5 = 2. \nFinally, 2 * 5 = 10.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 130, 'total_tokens': 166, 'completion_time': 0.130819931, 'prompt_time': 0.014718956, 'queue_time': 0.057693706000000004, 'total_time': 0.145538887}, 'model_name': 'llama-3.2-90b-vision-preview', 'system_fingerprint': 'fp_07b97e5459', 'finish_reason': 'stop', 'logprobs': None} id='run-35cb89fb-1442-47e5-9633-ad9631aaeb47-0' usage_metadata={'input_tokens': 130, 'output_tokens': 36, 'total_tokens': 166}
First, 4 + 3 = 7. 
Then, 7 - 5 = 2. 
Finally, 2 * 5 = 10.


In [59]:
import re

# Define the addition function
def add_two_numbers(a: int, b: int) -> int:
    """
    Add two numbers.

    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The sum of the two numbers.
    """
    return a + b

# Define the multiplication function
def multiply_two_numbers(a: int, b: int) -> int:
    """
    Multiply two numbers.

    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The product of the two numbers.
    """
    return a * b

# Function to process the user's prompt
def process_prompt(prompt: str, stored_number: int = None, operation: str = None):
    # Extract numbers from the prompt
    numbers = re.findall(r'-?\d+', prompt)  # Handle negative numbers
    numbers = [int(num) for num in numbers]

    # Determine the operation based on the prompt
    if "add" in prompt.lower() or "sum" in prompt.lower():
        operation = "add"
    elif "multiply" in prompt.lower() or "product" in prompt.lower():
        operation = "multiply"

    if stored_number is not None:
        # If a number is already stored, use it as the first number
        if len(numbers) == 1:
            # Perform the operation with the stored number and the new number
            if operation == "add":
                result = add_two_numbers(stored_number, numbers[0])
                return f"The sum of {stored_number} and {numbers[0]} is {result}.", None, None
            elif operation == "multiply":
                result = multiply_two_numbers(stored_number, numbers[0])
                return f"The product of {stored_number} and {numbers[0]} is {result}.", None, None
            else:
                return "Please specify the operation (add or multiply).", stored_number, operation
        else:
            # If no new number is provided, prompt for the second number
            return f"You provided the number {stored_number}. Please provide the second number.", stored_number, operation
    else:
        # If no number is stored, check the current input
        if len(numbers) == 2:
            # If two numbers are found, perform the operation
            if operation == "add":
                result = add_two_numbers(numbers[0], numbers[1])
                return f"The sum of {numbers[0]} and {numbers[1]} is {result}.", None, None
            elif operation == "multiply":
                result = multiply_two_numbers(numbers[0], numbers[1])
                return f"The product of {numbers[0]} and {numbers[1]} is {result}.", None, None
            else:
                return "Please specify the operation (add or multiply).", None, operation
        elif len(numbers) == 1:
            # If one number is found, store it and prompt for the second number
            return f"You provided the number {numbers[0]}. Please provide the second number.", numbers[0], operation
        else:
            # If no numbers are found, prompt the user for both numbers
            return "Please provide two numbers to perform the operation.", None, operation

# Main interaction loop
def main():
    # Initialize stored_number and operation
    stored_number = None
    operation = None

    while True:
        # Get user input
        user_input = input("User: ")

        # Process the prompt
        response_content, stored_number, operation = process_prompt(user_input, stored_number, operation)
        
        # Display the AI's response
        print(f"Assistant: {response_content}")

        # If the AI is prompting for more information, continue the loop
        if "Please provide" not in response_content:
            break

if __name__ == "__main__":
    main()


Assistant: You provided the number 5. Please provide the second number.
Assistant: The sum of 5 and 5 is 10.


In [54]:
# Example usage
# calculate("What is 121 + 5 then multiply the result by 5 and finally result -5?")  # Both numbers provided
# calculate("Add 10 * 5")  # Only one number provided
calculate("what is sum of 15")  # Second number provided
# calculate("Multiply 20")  # Only one number provided
# calculate("4")  # Second number provided

AI Message: content='' additional_kwargs={'tool_calls': [{'id': 'call_exc0', 'function': {'arguments': '{"a": "0", "b": "15"}', 'name': 'add'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 857, 'total_tokens': 876, 'completion_time': 0.104171734, 'prompt_time': 0.15094006, 'queue_time': -0.21946186299999998, 'total_time': 0.255111794}, 'model_name': 'llama-3.2-90b-vision-preview', 'system_fingerprint': 'fp_07b97e5459', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-6cc72b94-95bf-47e3-b829-c827af221448-0' tool_calls=[{'name': 'add', 'args': {'a': '0', 'b': '15'}, 'id': 'call_exc0', 'type': 'tool_call'}] usage_metadata={'input_tokens': 857, 'output_tokens': 19, 'total_tokens': 876}
Tool Calls: [{'name': 'add', 'args': {'a': '0', 'b': '15'}, 'id': 'call_exc0', 'type': 'tool_call'}]
Final Response: The sum of 15 is not a calculation but a single number, 15.


In [9]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

In [10]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. You may not need to use tools for every query - the user may just want to chat!",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [11]:
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [12]:
from langchain_core.messages import HumanMessage

agent_executor.invoke({"messages": [HumanMessage(content="what is the sum of 5")]})



> Entering new AgentExecutor chain...
I need two numbers to calculate the sum, but you've only provided one. Could you please provide the other number you'd like to add to 5?

> Finished chain.


{'messages': [HumanMessage(content='what is the sum of 5', additional_kwargs={}, response_metadata={})],
 'output': "I need two numbers to calculate the sum, but you've only provided one. Could you please provide the other number you'd like to add to 5?"}

In [18]:
from langchain_core.messages import HumanMessage

agent_executor.invoke({"messages": [HumanMessage(content="what is sum of 4,35,5 then multiply the result  of it by 6")]})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 4, 'b': 35}`


39
Invoking: `add` with `{'a': 39, 'b': 5}`


44
Invoking: `multiply` with `{'a': 44, 'b': 6}`


264First, the sum of 4 and 35 is 39. Then, adding 5 to that makes 44. Finally, multiplying 44 by 6 results in 264.

> Finished chain.


{'messages': [HumanMessage(content='what is sum of 4,35,5 then multiply the result  of it by 6', additional_kwargs={}, response_metadata={})],
 'output': 'First, the sum of 4 and 35 is 39. Then, adding 5 to that makes 44. Finally, multiplying 44 by 6 results in 264.'}

In [14]:
agent_executor.invoke({"messages": [HumanMessage(content="Add 121 to 5 then multiply result by 2")]})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 121, 'b': 5}`


126
Invoking: `multiply` with `{'a': 126, 'b': 2}`


252First, the function `add` was used to add 121 and 5, resulting in 126. Then, the function `multiply` was used to multiply the result by 2, resulting in 252.

> Finished chain.


{'messages': [HumanMessage(content='Add 121 to 5 then multiply result by 2', additional_kwargs={}, response_metadata={})],
 'output': 'First, the function `add` was used to add 121 and 5, resulting in 126. Then, the function `multiply` was used to multiply the result by 2, resulting in 252.'}